<!-- dom:TITLE: Varmeligning og differansemetoder. -->
# Varmeligning og differansemetoder.
<!-- dom:AUTHOR: Anne Kværnø, modified by André Massing -->
<!-- Author: -->  
**Anne Kværnø, modified by André Massing, A. Schmeding**

Date: **Jun 18, 2025**

Last inn nødvendige moduler

In [ ]:
%matplotlib inline

import numpy as np
from numpy import pi
from numpy.linalg import solve, norm    
import matplotlib.pyplot as plt

# Use a funny plotting style
plt.xkcd()
newparams = {'figure.figsize': (6.0, 6.0), 'axes.grid': True,
             'lines.markersize': 8, 'lines.linewidth': 2,
             'font.size': 14}
plt.rcParams.update(newparams)

# Introduction
In this note the finite difference method for solving partial differential equations
(PDEs) will be presented. 

Roughly speaking, a finite difference method is composed by the following steps: 
1. Discretize the domain on which the equation is defined.  

2. On each grid point, replace the derivatives with an approximation, using the values in neighbouring grid points. 

3. Replace the exact solutions by its approximations.

4. Solve the resulting system of equations. 

We will first see how to find approximations to the
derivative of a function, and then how this can be used to solve boundary value
problems like

# Varmeligning

I denne seksjonen skal vi se hvordan man kan løse varmeligningen ved hjelp av endelige differansemetoder. Det bør imidlertid understrekes at strategiene gjelder for mange tidsavhengige PDE-er — varmeligningen er bare et eksempel.

Husk varmeligningen i 1D:

$$
\begin{align*}
  u_t & = u_{xx}, && 0 \leq x \leq 1 \\ 
  u(0,t) &=g_0(t), \quad u(1,t)=g_1(t), && \text{Rand betringelser} \\ 
  u(x,0) &= f(x) && \text{Initial betingelser}
\end{align*}
$$

Ligningen løses fra $t=0$ til $t=t_{end}$. 

## Semi-diskretisering

Vi ser nå på en teknikk som kombinerer diskretisering av randverdiproblemer, med teknikker for løsning av ordinære differensialligninger.

Ideen er som følger

**Steg 1:**
Lag en diskretisering av intervalet i $x$-retning. Velg $M$ og la $\Delta x=1/M$ (siden vi jobber med interval $[0,1]$) og gitter punkter $x_i=i\Delta x$, $i=0,1,\dotsc,M$. 

Husk at for hver puntk $x_i$ løsningen $u(x_i,t)$ er en funksjon i $t$. 

**Steg 2:**
Fikser et tidspunkt $t$, og diskretiser høyre side av PDE-en. Bruk sentrale differanser for å approksimere $u_{xx}$, det gir oss

$$
\frac{\partial u}{\partial t}(x_i,t) =
\frac{u(x_{i+1},t)-2u(x_i,t)+u(x_{i-1},t)}{\Delta x^2}.
$$

**Steg 3:** 
Bytt $u(x_i,t)$ wimed approksimasjonen $U_i(t)$ i formelen. Dette gir oss

$$
U'_i(t) = \frac{U_{i+1}(t) - 2 U_i(t) + U_{i-1}(t)}{\Delta x^2}, \qquad i=1,2,\dotsc,M-1,
$$

med $U'_i(t) = dU_i(t)/dt$. 
Sammen med rand betingelser $U_0(t)=g_0(t)$, $U_M(t)=g_1(t)$ og intial betingelser $U_i(0) = f(x_i)$, $i=0,1,\dotsc,M$ får vi et system av ordinære differensialligninger

Systemet kalles vanligvis en *semi-diskretisering* av PDE-en. 

**Steg 4:**
Løs ODE-er med en numerisk metode. 

Vi bruker her eksplisitt Euler metode med steglengde $\Delta t$ for ODE-ene:

$$
U_{i}^{n+1} = U_{i}^n + r \big(\; U_{i+1}^n - 2U_{i}^n
+ U_{i-1}^n\; \big), \quad i=1,2,\dotsc,M-1, \qquad \text{med } r = \frac{\Delta t}{\Delta x^2}.
$$

Dermed $U_i^n \approx u(x_i,t_n)$ hvor $t_n=n\Delta t$, og for å skille indekser som representerer tid og rom bruker vi tidsindeks $n$ som superscripts. 

La oss teste algoritmen i to eksempler.

**Numerisk eksempel 1:** 
Løs varmeligning $u_t=u_{xx}$ på intervalet $0<t<1$ slik at:

$$
\begin{align*}
u(x,0) & = \sin(\pi x), && \text{Initial verdi} \\ 
g_0(t) & =g_1(t)=0. && \text{Rand betingelse}
\end{align*}
$$

Bruk steglengde $\Delta t=1/N$ og $\Delta x=1/M$. 

Eksakt løsning til problemet er gitt som

$$
u(x,t) = e^{-\pi^2t}\sin(\pi x).
$$

**Numerisk eksempel 2:**
Gjenta eksempel 1, men med intial verdier

$$
u(x,0) = \left\{ \begin{array}{ll} 2x & 0 \leq x \leq 0.5, \\ 2(1-x) & 0.5 < x\leq 2-2x, 
\end{array} \right.
$$

For dette gir vi ikke en eksakt løsning.

Kjør koden nede med 
1. $M=4$, $N=20$.

2. $M=8$, $N=40$.

3. $M=16$, $N=80$. 

Begge initialverdier er implementert. 

## Implementering

WFørst lager vi en funksjon som plotter løsningen.

In [ ]:
def plot_heat_solution(x, t, U, txt='Solution'):
    # Help function
    # Plot the solution of the heat equation
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    T, X = np.meshgrid(t,x)
    # ax.plot_wireframe(T, X, U)
    ax.plot_surface(T, X, U, cmap=plt.cm.coolwarm)
    ax.view_init(azim=30)              # Rotate the figure
    plt.xlabel('t')
    plt.ylabel('x')
    plt.title(txt);

Definer problemet, med intiial og rand betingelser.

In [ ]:
    # Define the problem

    # Initial condition
    def f1(x):               # Example 1
        return np.sin(pi*x)

    def f2(x):               # Example 2
        y = 2*x
        y[x>0.5] = 2-2*x[x>0.5]
        return y

    f = f1
    
    # Boundary conditions
    def g0(t):
        return 0
    def g1(t):
        return 0

Hoveddel av koden er:

In [ ]:
# Solve the heat equation by a forward difference in time (forward Euler)
#
M = 4                   # Number of intervals in the x-direction
Dx = 1/M
x = np.linspace(0,1,M+1)   # Gridpoints in the x-direction

tend = 0.5
N = 20                  #  Number of intervals in the t-direction
Dt = tend/N
t = np.linspace(0,tend,N+1) # Gridpoints in the t-direction

# Array to store the solution
U = np.zeros((M+1,N+1))
U[:,0] = f(x)              # Initial condition U_{i,0} = f(x_i)

r = Dt/Dx**2                
print('r =',r)

# Main loop 
for n in range(N):
    U[1:-1, n+1] = U[1:-1,n] + r*(U[2:,n]-2*U[1:-1,n]+U[0:-2,n])  
    U[0, n+1] = g0(t[n+1])
    U[M, n+1] = g1(t[n+1])

In [ ]:
# Plot the numerical solution
plot_heat_solution(x, t, U)

In [ ]:
# Plot the error from example 1
def u_exact(x,t):
    return np.exp(-pi**2*t)*np.sin(pi*x)
T, X = np.meshgrid(t, x)
error = u_exact(X, T) - U
plot_heat_solution(x, t, error, txt='Error')
print('Maximum error: {:.3e}'.format(max(abs(error.flatten()))))       # Maksimal feil over hele arrayet

Løsning er stabil for $M=4$, $N=20$, og ikke stabil for $M=16$, $N=80$. 
Hvorfor?  

## Bonus: Stability analysis

The semi-discretized system

$$
\dot{U}_i(t) = \frac{U_{i+1}(t) - 2 U_i(t) + U_{i-1}(t)}{\Delta x^2}, \qquad i=1,2,\dotsc,M-1, \qquad U_0(t)=g_0(t), \qquad U_M(t)=g_1(t),
$$

is a linear ordinary differential equation:

$$
\dot{\mathbf{U}} =  \frac{1}{\Delta x^2}\big( A \mathbf{U} + \mathbf{g}(t)\big),
$$

where

$$
\mathbf{U} = \left(\begin{array}{c} U_1 \\ U_2 \\ \vdots \\ U_{M-1} \end{array} \right), \qquad
A = \left(\begin{array}{cccc}
      -2 & 1 & \\ 
      1 & \ddots & \ddots \\ 
        &  \ddots & \ddots & 1 \\ 
        &  & 1 & -2
    \end{array}\right)
\qquad \text{and} \qquad
\mathbf{g}(t) = \left(\begin{array}{c} g_0(t) \\ 0 \\ \vdots \\ 0 \\ g_1(t) \end{array} \right)
$$

Stability requirements for such problems were discussed in the note on stiff ordinary differential equation. We proved there that the stability depends on the eigenvalues $\lambda_k$ of the matrix $\frac{1}{\Delta x^2}A$. For the forward Euler method, it was proved that the step size has to chosen such that $-2 \leq \Delta t \lambda_k \leq 0$ for all $\lambda_k$. Otherwise, the numerical solution will be unstable.

It is possible to prove that the eigenvalues of the matrix $A$ is given by

$$
\lambda_k = -4\sin^2 \big( \frac{k\pi}{M}\big), \qquad k=1,\cdots,M-1.
$$

So all the eigenvalues $\lambda_k$ of $\frac{1}{\Delta x^2}A$ satisfy

$$
-\frac{4}{\Delta x^2} < \lambda _k < 0.
$$

The numerical solution is stable if $\Delta t < -2/\lambda_k$ for all $k$, that means whenever

$$
r = \frac{\Delta t}{\Delta x^2} \leq \frac{1}{2}.
$$

**Exercise:**
Repeat the two experiments above (for the two different initial values) to justify the bound above.  
Use $M=16$, and in each case find the corresponding $r$ and observe from the experiments whether the solution is stable or not. 

1. Let $N=256$.

2. Let $N=128$.  

3. Let $N=250$. 

In the last case, it seems like the metod is unstable for the first initial value, and unstable for the second. Do you have any idea why? (Both solutions will be unstable if integrated over a longer time periode). 


## Implicit methods

The semi-discretized system is an example of a stiff ODE, which can only be handled reasonable efficiently by $A(0)$-stable methods, like implicit Euler or the trapezoidal rule. 